# OpenADMET Multi-Task Training with Graph Transformer
This notebook demonstrates how to train a **Graph Transformer** model on the
[OpenADMET](https://openadmet.ghost.io/openadmet-expansionrx-blind-challenge/) multi-task dataset using the `gt-pyg` library.

- Graph Transformer in Pytorch Geometric[gt_pyg](https://github.com/pgniewko/gt-pyg)
- `v1.6.0.0`

> **Note:** This is an exemplary training run for demonstration purposes,
> not a production-grade model.

**Protocol summary:**
- 9 ADMET endpoints trained jointly (sparse labels — not every molecule has every endpoint)
- Single loss term: **MAE** (no weighting, no rescaling)
- 250 epochs, **cosine annealing** LR schedule (no warmup)
- Track best macro-averaged MAE on the validation set
- Evaluate on the held-out test set (public leaderboard + private split)

> **Loss caveat:** The plain MAE loss treats all endpoints and all datapoints
> uniformly. Endpoints with more observed labels or larger variance therefore
> have disproportionate influence on the gradient. A production model would
> use per-task normalisation (e.g. RAE) or a multi-objective loss.

## 1. Imports

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch_geometric.loader import DataLoader
from scipy.stats import spearmanr, kendalltau
from sklearn.metrics import r2_score
from copy import deepcopy

from gt_pyg.data import get_tensor_data, get_atom_feature_dim, get_bond_feature_dim
from gt_pyg.nn import GraphTransformerNet

## 2. Configuration

In [25]:
# Paths
TRAIN_CSV = "data/train-set/expansion_log_data_train.csv"
TEST_CSV = "data/test-set/expansion_data_test_full_lb_flag.csv"

# Model
HIDDEN_DIM = 128
NUM_GT_LAYERS = 4
NUM_HEADS = 8
DROPOUT = 0.1

# Training
EPOCHS = 250
BATCH_SIZE = 256
LR = 1e-3
WEIGHT_DECAY = 1e-5

SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"Device: {DEVICE}")

Device: cpu


## 3. Load and Split Data

The training set contains log-transformed ADMET endpoint values. Many values are
missing — this is a **sparse multi-task** problem. We perform a random 80/20
train/validation split.

In [26]:
log_train_df = pd.read_csv(TRAIN_CSV)

ID_COLS = {"SMILES", "Molecule Name"}
ENDPOINTS = [c for c in log_train_df.columns if c not in ID_COLS]
NUM_TASKS = len(ENDPOINTS)

print(f"Total molecules: {len(log_train_df)}")
print(f"Endpoints ({NUM_TASKS}): {ENDPOINTS}")
print(f"\nLabel coverage per endpoint:")
for ep in ENDPOINTS:
    n = log_train_df[ep].notna().sum()
    print(f"  {ep}: {n} / {len(log_train_df)} ({100*n/len(log_train_df):.1f}%)")

Total molecules: 5326
Endpoints (9): ['LogD', 'LogS', 'Log_HLM_CLint', 'Log_MLM_CLint', 'Log_Caco_Papp_AB', 'Log_Caco_ER', 'Log_Mouse_PPB', 'Log_Mouse_BPB', 'Log_Mouse_MPB']

Label coverage per endpoint:
  LogD: 5039 / 5326 (94.6%)
  LogS: 5128 / 5326 (96.3%)
  Log_HLM_CLint: 3759 / 5326 (70.6%)
  Log_MLM_CLint: 4522 / 5326 (84.9%)
  Log_Caco_Papp_AB: 2157 / 5326 (40.5%)
  Log_Caco_ER: 2161 / 5326 (40.6%)
  Log_Mouse_PPB: 1302 / 5326 (24.4%)
  Log_Mouse_BPB: 975 / 5326 (18.3%)
  Log_Mouse_MPB: 222 / 5326 (4.2%)


In [27]:
# Shuffle and split 80/20
df = log_train_df.sample(frac=1, random_state=SEED).reset_index(drop=True)
n_train = int(0.8 * len(df))
tr_df = df.iloc[:n_train].copy()
va_df = df.iloc[n_train:].copy()

print(f"Train: {len(tr_df)}, Validation: {len(va_df)}")

Train: 4260, Validation: 1066


## 4. Build PyG Datasets and DataLoaders

`get_tensor_data` converts SMILES + labels into PyG `Data` objects with
atom/bond features, GNM positional encodings, and multi-task label tensors.
Missing labels are encoded as `NaN` with an accompanying binary mask (`y_mask`).

In [28]:
def build_dataset(df, endpoints):
    """Convert a DataFrame into a list of PyG Data objects."""
    smiles = df["SMILES"].tolist()
    labels = df[endpoints].values.tolist()  # list of lists, NaN for missing
    return get_tensor_data(smiles, labels)

print("Building training set...")
tr_dataset = build_dataset(tr_df, ENDPOINTS)
print("Building validation set...")
va_dataset = build_dataset(va_df, ENDPOINTS)

tr_loader = DataLoader(tr_dataset, batch_size=BATCH_SIZE, shuffle=True)
va_loader = DataLoader(va_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"\nTrain batches: {len(tr_loader)}, Val batches: {len(va_loader)}")
print(f"Node features: {tr_dataset[0].x.shape[1]}, Edge features: {tr_dataset[0].edge_attr.shape[1]}")

Building training set...


Processing data:  14%|█████████████▋                                                                                     | 591/4260 [00:00<00:05, 624.38it/s]/Users/pawelgniewek/projects/gt-pyg/gt_pyg/data/utils.py:236: RuntimeWarning: divide by zero encountered in divide
  inv_eigenvalues = np.where(np.abs(eigenvalues) > 1e-10, 1.0 / eigenvalues, 0.0)
Processing data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4260/4260 [00:06<00:00, 665.60it/s]


Building validation set...


Processing data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1066/1066 [00:01<00:00, 649.85it/s]


Train batches: 17, Val batches: 5
Node features: 139, Edge features: 39


## 5. Create Model, Optimizer, and Scheduler

We use `GraphTransformerNet` with a variational readout head. During evaluation
the model returns the deterministic mean (`mu`). The LR follows a **cosine
annealing** schedule from `LR` down to 0 over 250 epochs.

In [29]:
node_dim = get_atom_feature_dim()
edge_dim = get_bond_feature_dim()

model = GraphTransformerNet(
    node_dim_in=node_dim,
    edge_dim_in=edge_dim,
    hidden_dim=HIDDEN_DIM,
    num_gt_layers=NUM_GT_LAYERS,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    num_tasks=NUM_TASKS,
).to(DEVICE)

print(f"Parameters: {model.num_parameters():,}")

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=0)

Parameters: 2,564,914


## 6. Training and Evaluation Functions

The loss is **masked MAE**: for each sample, only endpoints with observed labels
contribute to the loss. This handles the sparse multi-task structure naturally.

Evaluation computes the **5 official challenge metrics** per endpoint:
MAE, RAE, R², Spearman $\rho$, Kendall $\tau$.

In [30]:
def masked_mae_loss(pred, target, mask):
    """MAE loss computed only over observed (non-NaN) labels."""
    diff = torch.abs(pred - target) * mask
    count = mask.sum()
    if count == 0:
        return torch.tensor(0.0, device=pred.device)
    return diff.sum() / count


def train_epoch(model, loader, optimizer, device):
    """Run one training epoch. Returns average loss."""
    model.train()
    total_loss = 0.0
    n_batches = 0

    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        pred, _ = model(
            x=batch.x,
            edge_index=batch.edge_index,
            edge_attr=batch.edge_attr,
            batch=batch.batch,
        )

        target = batch.y
        mask = batch.y_mask
        # Replace NaN in target with 0 to avoid NaN in loss
        target = torch.nan_to_num(target, nan=0.0)

        loss = masked_mae_loss(pred, target, mask)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        n_batches += 1

    return total_loss / max(n_batches, 1)


def _official_metrics(y_true_1d, y_pred_1d):
    """Compute the 5 official challenge metrics for a single endpoint."""
    y = np.asarray(y_true_1d).ravel()
    p = np.asarray(y_pred_1d).ravel()
    m = np.isfinite(y) & np.isfinite(p)
    y = y[m]
    p = p[m]
    if y.size == 0:
        return {
            "MAE": np.nan,
            "RAE": np.nan,
            "R2": np.nan,
            "Spearman R": np.nan,
            "Kendall's Tau": np.nan,
        }
    mae = float(np.mean(np.abs(y - p)))
    denom = np.mean(np.abs(y - np.mean(y))) if y.size > 0 else np.nan
    rae = float(mae / denom) if denom and np.isfinite(denom) and denom > 0 else np.nan
    r2 = float(r2_score(y, p)) if (np.nanstd(y) > 0) else np.nan
    if np.nanstd(p) < 1e-4:
        spr = np.nan
        ktau = np.nan
    else:
        spr = float(spearmanr(y, p)[0])
        ktau = float(kendalltau(y, p)[0])
    return {
        "MAE": mae,
        "RAE": rae,
        "R2": r2,
        "Spearman R": spr,
        "Kendall's Tau": ktau,
    }


@torch.no_grad()
def evaluate(model, loader, device, endpoints):
    """Evaluate model. Returns (metrics_dict, preds, targets, masks).

    metrics_dict[endpoint] has 5 official metrics + N.
    metrics_dict["Average"] has macro-averaged values (nanmean).
    """
    model.eval()
    all_preds = []
    all_targets = []
    all_masks = []

    for batch in loader:
        batch = batch.to(device)
        pred, _ = model(
            x=batch.x,
            edge_index=batch.edge_index,
            edge_attr=batch.edge_attr,
            batch=batch.batch,
        )
        all_preds.append(pred.cpu())
        all_targets.append(batch.y.cpu())
        all_masks.append(batch.y_mask.cpu())

    preds = torch.cat(all_preds, dim=0).numpy()
    targets = torch.cat(all_targets, dim=0).numpy()
    masks = torch.cat(all_masks, dim=0).numpy()

    metrics = {}
    for i, ep in enumerate(endpoints):
        valid = masks[:, i] > 0.5
        if valid.sum() < 2:
            continue
        y = targets[valid, i]
        p = preds[valid, i]
        official = _official_metrics(y, p)
        official["N"] = int(valid.sum())
        metrics[ep] = official

    # Macro averages (nanmean over endpoints)
    keys = ["MAE", "RAE", "R2", "Spearman R", "Kendall's Tau"]
    avg = {}
    for k in keys:
        vals = [metrics[ep][k] for ep in metrics if ep != "Average"]
        avg[k] = float(np.nanmean(vals)) if vals else np.nan
    metrics["Average"] = avg

    return metrics, preds, targets, masks

## 7. Training Loop

We train for 250 epochs, tracking the **best macro-averaged MAE** on the
validation set. The best model weights are stored in memory.

In [ ]:
best_val_mae = float("inf")
best_model_state = None
best_epoch = -1

history = {
    "epoch": [], "train_loss": [], "lr": [],
    "val_mae": [], "val_rae": [], "val_r2": [],
    "val_spearman": [], "val_kendall": [],
    # Train metrics recorded at logged epochs only
    "train_mae": [], "train_rae": [], "train_r2": [],
    "train_spearman": [], "train_kendall": [],
    "logged_epoch": [],
}

KT_KEY = "Kendall's Tau"

for epoch in range(1, EPOCHS + 1):
    train_loss = train_epoch(model, tr_loader, optimizer, DEVICE)
    scheduler.step()

    val_metrics_ep, _, _, _ = evaluate(model, va_loader, DEVICE, ENDPOINTS)

    val_avg = val_metrics_ep["Average"]
    val_mae = val_avg["MAE"]
    lr_now = optimizer.param_groups[0]["lr"]

    history["epoch"].append(epoch)
    history["train_loss"].append(train_loss)
    history["lr"].append(lr_now)
    history["val_mae"].append(val_mae)
    history["val_rae"].append(val_avg["RAE"])
    history["val_r2"].append(val_avg["R2"])
    history["val_spearman"].append(val_avg["Spearman R"])
    history["val_kendall"].append(val_avg[KT_KEY])

    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_epoch = epoch
        best_model_state = deepcopy(model.state_dict())

    if epoch % 10 == 0 or epoch == 1:
        tr_metrics_ep, _, _, _ = evaluate(model, tr_loader, DEVICE, ENDPOINTS)
        tr_avg = tr_metrics_ep["Average"]

        history["logged_epoch"].append(epoch)
        history["train_mae"].append(tr_avg["MAE"])
        history["train_rae"].append(tr_avg["RAE"])
        history["train_r2"].append(tr_avg["R2"])
        history["train_spearman"].append(tr_avg["Spearman R"])
        history["train_kendall"].append(tr_avg[KT_KEY])

        print(
            f"Epoch {epoch:3d}/{EPOCHS} | "
            f"Loss: {train_loss:.4f} | "
            f"Train MAE={tr_avg['MAE']:.3f} RAE={tr_avg['RAE']:.3f} "
            f"R2={tr_avg['R2']:.3f} rho={tr_avg['Spearman R']:.3f} "
            f"tau={tr_avg[KT_KEY]:.3f} | "
            f"Val MAE={val_avg['MAE']:.3f} RAE={val_avg['RAE']:.3f} "
            f"R2={val_avg['R2']:.3f} rho={val_avg['Spearman R']:.3f} "
            f"tau={val_avg[KT_KEY]:.3f} | "
            f"LR: {lr_now:.2e}"
        )

print(f"\nBest val MAE: {best_val_mae:.4f} at epoch {best_epoch}")

Epoch   1/250 | Loss: 1.0036 | Train MAE=0.489 RAE=1.037 R2=-0.145 rho=0.111 tau=0.074 | Val MAE=0.492 RAE=1.032 R2=-0.131 rho=0.110 tau=0.075 | LR: 1.00e-03


## 8. Restore Best Model and Print Final Stats

Load the best checkpoint (by validation MAE) and report per-endpoint metrics
on both train and validation sets.

In [ ]:
model.load_state_dict(best_model_state)

tr_metrics, _, _, _ = evaluate(model, tr_loader, DEVICE, ENDPOINTS)
va_metrics, _, _, _ = evaluate(model, va_loader, DEVICE, ENDPOINTS)

header = (
    f"{'Endpoint':<22} "
    f"{'Train MAE':>10} {'Val MAE':>10} "
    f"{'Train RAE':>10} {'Val RAE':>10} "
    f"{'Train R2':>10} {'Val R2':>10} "
    f"{'Train rho':>10} {'Val rho':>10} "
    f"{'Train tau':>10} {'Val tau':>10} "
    f"{'N_val':>7}"
)
print(header)
print("-" * len(header))

KT_KEY = "Kendall's Tau"

for ep in ENDPOINTS:
    if ep in tr_metrics and ep in va_metrics:
        tm = tr_metrics[ep]
        vm = va_metrics[ep]
        print(
            f"{ep:<22} "
            f"{tm['MAE']:>10.4f} {vm['MAE']:>10.4f} "
            f"{tm['RAE']:>10.4f} {vm['RAE']:>10.4f} "
            f"{tm['R2']:>10.4f} {vm['R2']:>10.4f} "
            f"{tm['Spearman R']:>10.4f} {vm['Spearman R']:>10.4f} "
            f"{tm[KT_KEY]:>10.4f} {vm[KT_KEY]:>10.4f} "
            f"{vm['N']:>7d}"
        )

print("-" * len(header))
ta = tr_metrics["Average"]
va = va_metrics["Average"]
print(
    f"{'Average':<22} "
    f"{ta['MAE']:>10.4f} {va['MAE']:>10.4f} "
    f"{ta['RAE']:>10.4f} {va['RAE']:>10.4f} "
    f"{ta['R2']:>10.4f} {va['R2']:>10.4f} "
    f"{ta['Spearman R']:>10.4f} {va['Spearman R']:>10.4f} "
    f"{ta[KT_KEY]:>10.4f} {va[KT_KEY]:>10.4f}"
)

## 9. Training Curves

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# ---- (0,0) Loss curves (LOG SCALE) ----
ax = axes[0, 0]
ax.semilogy(history["epoch"], history["train_loss"], label="Train Loss", alpha=0.8)
# Compute val loss from val_mae as proxy (we don't track val loss separately)
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss (log scale)")
ax.set_title("Training Loss")
ax.legend()
ax.grid(True, alpha=0.3)

# ---- (0,1) RAE (capped at 1.0) ----
ax = axes[0, 1]
ax.plot(history["epoch"], history["val_rae"], "b-", alpha=0.8, label="Val RAE")
if history["logged_epoch"]:
    ax.plot(history["logged_epoch"], history["train_rae"], "b--", alpha=0.5, label="Train RAE")
best_rae = min(history["val_rae"])
ax.axhline(best_rae, color="r", linestyle="--", alpha=0.5, label=f"Best Val: {best_rae:.3f}")
ax.set_ylim(0, 1.0)
ax.set_xlabel("Epoch")
ax.set_ylabel("RAE")
ax.set_title("RAE (capped at 1.0)")
ax.legend()
ax.grid(True, alpha=0.3)

# ---- (1,0) R² (capped at 1.0) ----
ax = axes[1, 0]
ax.plot(history["epoch"], history["val_r2"], "g-", alpha=0.8, label="Val R²")
if history["logged_epoch"]:
    ax.plot(history["logged_epoch"], history["train_r2"], "g--", alpha=0.5, label="Train R²")
best_r2 = max(history["val_r2"])
ax.axhline(best_r2, color="r", linestyle="--", alpha=0.5, label=f"Best Val: {best_r2:.3f}")
ax.set_ylim(-0.1, 1.0)
ax.set_xlabel("Epoch")
ax.set_ylabel("R²")
ax.set_title("R² (capped at 1.0)")
ax.legend()
ax.grid(True, alpha=0.3)

# ---- (1,1) Kendall τ ----
ax = axes[1, 1]
ax.plot(history["epoch"], history["val_kendall"], "m-", alpha=0.8, label="Val τ")
if history["logged_epoch"]:
    ax.plot(history["logged_epoch"], history["train_kendall"], "m--", alpha=0.5, label="Train τ")
best_tau = max(history["val_kendall"])
ax.axhline(best_tau, color="r", linestyle="--", alpha=0.5, label=f"Best Val: {best_tau:.3f}")
ax.set_xlabel("Epoch")
ax.set_ylabel("Kendall τ")
ax.set_title("Kendall τ")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Bar Plots — Train vs Validation MAE and R²

In [ ]:
# Build metrics dataframe for bar plots
# Use the endpoint keys from the metrics dicts directly, so this cell works
# regardless of whether the global ENDPOINTS has been overwritten.
_train_eps = [k for k in tr_metrics if k != "Average"]

metric_records = []
for ep in _train_eps:
    tm = tr_metrics.get(ep, {})
    vm = va_metrics.get(ep, {})
    metric_records.append({"Assay": ep, "Split": "Train", "R2": tm.get("R2", np.nan), "MAE": tm.get("MAE", np.nan)})
    metric_records.append({"Assay": ep, "Split": "Val", "R2": vm.get("R2", np.nan), "MAE": vm.get("MAE", np.nan)})

metrics_df = pd.DataFrame(metric_records)

# Order assays by validation MAE ascending
order_by_val = (
    metrics_df[metrics_df["Split"] == "Val"]
    .sort_values("MAE")["Assay"]
    .tolist()
)

sns.set_style("whitegrid")

# ---- MAE bar plot ----
fig, ax = plt.subplots(figsize=(12, 5))
sns.barplot(x="Assay", y="MAE", hue="Split", data=metrics_df, ax=ax, order=order_by_val)
ax.set_ylabel("MAE (log-space)")
ax.set_title("Training vs Validation MAE per Endpoint")
ax.set_xticklabels([t.get_text().replace("_", "\n") for t in ax.get_xticklabels()])
plt.tight_layout()
plt.show()

# ---- R² bar plot ----
fig, ax = plt.subplots(figsize=(12, 5))
sns.barplot(x="Assay", y="R2", hue="Split", data=metrics_df, ax=ax, order=order_by_val)
ax.set_ylim(-1.1, 1.0)
ax.set_ylabel("$R^2$")
ax.set_title("Training vs Validation R² per Endpoint")
ax.set_xticklabels([t.get_text().replace("_", "\n") for t in ax.get_xticklabels()])
plt.tight_layout()
plt.show()

## 11. Test Set Prediction & Submission

Generate predictions on the held-out test set and save as `submission.csv`.
The model predicts in **log-space** (matching the training labels), and we
inverse-transform to **original assay units** for the submission file.

In [ ]:
# --- Load test set ---
test_df = pd.read_csv(TEST_CSV)
print(f"Test samples: {len(test_df)}")
print(f"Columns: {test_df.columns.tolist()}")

smiles_test = test_df["SMILES"].tolist()

# --- Build PyG dataset (inference — no labels) ---
print("\nFeaturising test SMILES...")
test_dataset = get_tensor_data(smiles_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- Run inference ---
model.eval()
test_preds_list = []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(DEVICE)
        pred, _ = model(
            x=batch.x,
            edge_index=batch.edge_index,
            edge_attr=batch.edge_attr,
            batch=batch.batch,
        )
        test_preds_list.append(pred.cpu().numpy())

test_preds_log = np.concatenate(test_preds_list, axis=0)  # [N, 9] in log-space
print(f"Predictions shape: {test_preds_log.shape}")

# --- Inverse-transform from log-space to original assay units ---
# Forward transform was: log10((y + 1) * multiplier)
# Inverse:              (10^x / multiplier) - 1
# KSOL and Caco-2 Papp A>B have multiplier = 1e-6; all others have multiplier = 1
INVERSE_MAP = {
    "LogD":             ("LogD",                          lambda x: x),
    "LogS":             ("KSOL",                          lambda x: (10**x / 1e-6) - 1),
    "Log_HLM_CLint":    ("HLM CLint",                     lambda x: 10**x - 1),
    "Log_MLM_CLint":    ("MLM CLint",                     lambda x: 10**x - 1),
    "Log_Caco_Papp_AB": ("Caco-2 Permeability Papp A>B",  lambda x: (10**x / 1e-6) - 1),
    "Log_Caco_ER":      ("Caco-2 Permeability Efflux",    lambda x: 10**x - 1),
    "Log_Mouse_PPB":    ("MPPB",                          lambda x: 10**x - 1),
    "Log_Mouse_BPB":    ("MBPB",                          lambda x: 10**x - 1),
    "Log_Mouse_MPB":    ("MGMB",                          lambda x: 10**x - 1),
}

# Build submission DataFrame
sub_df = test_df[["Molecule Name", "SMILES"]].copy()

# Iterate over INVERSE_MAP keys (log-space names) — NOT the global ENDPOINTS,
# which may have been overwritten by the evaluation helpers cell.
for i, ep_log in enumerate(INVERSE_MAP):
    col_name, transform = INVERSE_MAP[ep_log]
    sub_df[col_name] = transform(test_preds_log[:, i])

sub_df.to_csv("submission.csv", index=False)
print(f"\nSaved submission.csv with {len(sub_df)} rows")
print(f"Columns: {sub_df.columns.tolist()}\n")

# --- Show head and summary ---
print("Head:")
print(sub_df.head().to_string())

print("\nPrediction range summary (original scale):")
for col in sub_df.columns:
    if col not in ("Molecule Name", "SMILES"):
        vals = sub_df[col]
        print(f"  {col:<35s}  min={vals.min():.4f}  max={vals.max():.4f}  mean={vals.mean():.4f}")

## 12. Test Set Evaluation — Leaderboard vs Private

Evaluate the submission against the full test-set ground truth, split three ways:

| Split | Description |
|-------|-------------|
| **LB** | Public leaderboard molecules (`is_leaderboard == 1`) |
| **Private** | Private test molecules (`is_leaderboard == 0`) |
| **All** | Full test set |

We report **RAE** (Relative Absolute Error) per endpoint and the **macro-averaged
RAE** (MA-RAE) across all 9 endpoints, with bootstrap confidence intervals.

In [ ]:
# ---------------------------------------------------------------------------
# Constants & evaluation helpers
# Verbatim from: constants.py, evaluate.py
# (matches OpenADMET/ExpansionRx-Challenge-Tutorial eval/ pipeline)
# ---------------------------------------------------------------------------

from typing import Tuple, Dict, Optional
from scipy.stats import spearmanr, kendalltau
from sklearn.metrics import mean_absolute_error, r2_score

# --- constants.py -----------------------------------------------------------
# Renamed to EVAL_ENDPOINTS to avoid overwriting the training ENDPOINTS
# (log-space names) defined in Cell 6 — which Cell 23 relies on.

EVAL_ENDPOINTS = [
    "LogD",
    "KSOL",
    "MLM CLint",
    "HLM CLint",
    "Caco-2 Permeability Efflux",
    "Caco-2 Permeability Papp A>B",
    "MPPB",
    "MBPB",
    "MGMB"
]

METRICS = ["MAE", "RAE", "R2", "Spearman R", "Kendall's Tau"]

# --- evaluate.py -------------------------------------------------------------


def clip_and_log_transform(y: np.ndarray) -> np.ndarray:
    """
    Clip to a detection limit and transform to log10 scale.

    Parameters
    ----------
    y : np.ndarray
        The array to be clipped and transformed.

    Returns
    -------
    np.ndarray
        The log10(y + 1) transformed array, with values clipped to >= 0.
    """
    y = np.clip(y, a_min=0, a_max=None)
    return np.log10(y + 1)


def bootstrap_sampling(size: int, n_samples: int, seed: int = 0) -> np.ndarray:
    """
    Generate bootstrap sample indices.

    Parameters
    ----------
    size : int
        The size of the original data.
    n_samples : int
        The number of bootstrap samples to generate.
    seed : int, optional
        Random seed for reproducibility, by default 0

    Returns
    -------
    np.ndarray
        Array of shape (n_samples, size) with bootstrap indices.
    """
    rng = np.random.default_rng(seed)
    return rng.choice(size, size=(n_samples, size), replace=True)


def metrics_per_ep(pred: np.ndarray, true: np.ndarray) -> Tuple[float, float, float, float, float]:
    """
    Calculate evaluation metrics for a single sample.

    Parameters
    ----------
    pred : np.ndarray
        Array with predictions
    true : np.ndarray
        Array with actual values

    Returns
    -------
    Tuple[float, float, float, float, float]
        Resulting metrics: (MAE, RAE, R2, Spearman R, Kendall's Tau)
    """
    mae = mean_absolute_error(true, pred)
    rae = mae / np.mean(np.abs(true - np.mean(true)))

    if np.nanstd(true) == 0:
        r2 = np.nan
    else:
        r2 = r2_score(true, pred)

    spr = spearmanr(true, pred).statistic
    ktau = kendalltau(true, pred).statistic

    return mae, rae, r2, spr, ktau


def bootstrap_metrics(
    pred: np.ndarray,
    true: np.ndarray,
    endpoint: str,
    n_bootstrap_samples: int = 1000
) -> pd.DataFrame:
    """
    Calculate bootstrap metrics given predicted and true values.

    Parameters
    ----------
    pred : np.ndarray
        Predicted endpoint values
    true : np.ndarray
        Actual endpoint values
    endpoint : str
        Name of the endpoint
    n_bootstrap_samples : int, optional
        Number of bootstrap samples, by default 1000

    Returns
    -------
    pd.DataFrame
        DataFrame with columns [Sample, Endpoint, Metric, Value]
        containing estimated metric per bootstrap sample for the given endpoint.
    """
    cols = ["Sample", "Endpoint", "Metric", "Value"]
    results = []

    for i, indx in enumerate(bootstrap_sampling(true.shape[0], n_bootstrap_samples)):
        mae, rae, r2, spr, ktau = metrics_per_ep(pred[indx], true[indx])
        results.extend([
            [i, endpoint, "MAE", mae],
            [i, endpoint, "RAE", rae],
            [i, endpoint, "R2", r2],
            [i, endpoint, "Spearman R", spr],
            [i, endpoint, "Kendall's Tau", ktau]
        ])

    return pd.DataFrame(results, columns=cols)


def _check_required_columns(df: pd.DataFrame, name: str, cols: list[str]) -> None:
    """Check that all required columns are present in the DataFrame."""
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"{name} is missing required columns: {missing}")


def calculate_metrics(
    predictions_df: pd.DataFrame,
    ground_truth_df: pd.DataFrame,
    n_bootstrap_samples: int = 1000,
    endpoints: Optional[list[str]] = None
) -> pd.DataFrame:
    """
    Calculate evaluation metrics for all endpoints.

    This is the main evaluation function that:
    1. Merges predictions with ground truth on 'Molecule Name'
    2. For each endpoint, applies log10(y+1) transform (except LogD)
    3. Calculates bootstrap metrics
    4. Returns summary statistics (mean +/- std) for each endpoint

    Parameters
    ----------
    predictions_df : pd.DataFrame
        DataFrame with predictions. Must have 'Molecule Name' column and
        one column per endpoint.
    ground_truth_df : pd.DataFrame
        DataFrame with ground truth values. Must have 'Molecule Name' column
        and one column per endpoint.
    n_bootstrap_samples : int, optional
        Number of bootstrap samples for confidence intervals, by default 1000
    endpoints : list[str], optional
        List of endpoints to evaluate. If None, uses all EVAL_ENDPOINTS.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns [Endpoint, mean_MAE, std_MAE, mean_RAE, std_RAE,
        mean_R2, std_R2, mean_Spearman R, std_Spearman R, mean_Kendall's Tau,
        std_Kendall's Tau]. Includes a 'Macro Average' row at the end.

    Raises
    ------
    ValueError
        If required columns are missing or data validation fails.
    """
    if endpoints is None:
        endpoints = EVAL_ENDPOINTS

    # Check required columns
    _check_required_columns(predictions_df, "Predictions file", ["Molecule Name"] + endpoints)
    _check_required_columns(ground_truth_df, "Ground truth file", ["Molecule Name"] + endpoints)

    # Check all molecules in predictions are in ground truth
    if not predictions_df['Molecule Name'].isin(ground_truth_df['Molecule Name']).all():
        raise ValueError("Predictions file contains molecules not in ground truth")

    # Check for duplicates in predictions
    if predictions_df['Molecule Name'].duplicated().any():
        raise ValueError("Predictions file contains duplicated molecules")

    # Merge dataframes
    merged_df = predictions_df.merge(
        ground_truth_df,
        on="Molecule Name",
        suffixes=('_pred', '_true'),
        how="inner"
    )
    merged_df = merged_df.sort_values("Molecule Name")

    all_endpoint_results = []

    for ept in endpoints:
        pred_col = f"{ept}_pred"
        true_col = f"{ept}_true"

        # Cast to numeric
        merged_df[pred_col] = pd.to_numeric(merged_df[pred_col], errors="coerce")
        merged_df[true_col] = pd.to_numeric(merged_df[true_col], errors="coerce")

        if merged_df[pred_col].isnull().all():
            raise ValueError(f"All predictions are missing for endpoint {ept}")

        # Subset and drop NaNs
        subset = merged_df[[pred_col, true_col]].dropna()
        if subset.empty:
            raise ValueError(f"No valid data available for endpoint {ept} after removing NaNs")

        # Extract numpy arrays
        y_pred = subset[pred_col].to_numpy()
        y_true = subset[true_col].to_numpy()

        # Apply log10(y+1) transform except for LogD
        if ept.lower() not in ['logd']:
            y_true_log = clip_and_log_transform(y_true)
            y_pred_log = clip_and_log_transform(y_pred)
        else:
            y_true_log = y_true
            y_pred_log = y_pred

        # Calculate bootstrap metrics
        bootstrap_df = bootstrap_metrics(y_pred_log, y_true_log, ept, n_bootstrap_samples)

        # Pivot to get mean and std
        df_endpoint = bootstrap_df.pivot_table(
            index=["Endpoint"],
            columns="Metric",
            values="Value",
            aggfunc=["mean", "std"]
        ).reset_index()

        # Flatten column names
        df_endpoint.columns = [
            f'{i}_{j}' if i != '' else j for i, j in df_endpoint.columns
        ]
        df_endpoint.rename(columns={'Endpoint_': 'Endpoint'}, inplace=True)

        all_endpoint_results.append(df_endpoint)

    # Combine all results
    df_results = pd.concat(all_endpoint_results, ignore_index=True)

    # Calculate macro average
    mean_cols = [f'mean_{m}' for m in METRICS]
    std_cols = [f'std_{m}' for m in METRICS]

    macro_means = df_results[mean_cols].mean()
    macro_stds = df_results[std_cols].mean()

    avg_row = {"Endpoint": "Macro Average"}
    avg_row.update(macro_means.to_dict())
    avg_row.update(macro_stds.to_dict())

    df_with_average = pd.concat([df_results, pd.DataFrame([avg_row])], ignore_index=True)

    # Fix column order
    df_with_average = df_with_average[["Endpoint"] + mean_cols + std_cols]

    return df_with_average


def get_point_predictions_with_truth(
    predictions_df: pd.DataFrame,
    ground_truth_df: pd.DataFrame,
    endpoint: str
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Get aligned prediction and ground truth arrays for an endpoint.

    Useful for scatter plots of predicted vs actual values.

    Parameters
    ----------
    predictions_df : pd.DataFrame
        DataFrame with predictions.
    ground_truth_df : pd.DataFrame
        DataFrame with ground truth values.
    endpoint : str
        Endpoint name.

    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        Tuple of (predictions, ground_truth) arrays (log-transformed if applicable).
    """
    merged_df = predictions_df.merge(
        ground_truth_df,
        on="Molecule Name",
        suffixes=('_pred', '_true'),
        how="inner"
    )

    pred_col = f"{endpoint}_pred"
    true_col = f"{endpoint}_true"

    merged_df[pred_col] = pd.to_numeric(merged_df[pred_col], errors="coerce")
    merged_df[true_col] = pd.to_numeric(merged_df[true_col], errors="coerce")

    subset = merged_df[[pred_col, true_col]].dropna()

    y_pred = subset[pred_col].to_numpy()
    y_true = subset[true_col].to_numpy()

    # Apply log10(y+1) transform except for LogD
    if endpoint.lower() not in ['logd']:
        y_true = clip_and_log_transform(y_true)
        y_pred = clip_and_log_transform(y_pred)

    return y_pred, y_true


def format_metrics_table(df: pd.DataFrame, precision: int = 3) -> pd.DataFrame:
    """
    Format the metrics DataFrame with mean +/- std format.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame from calculate_metrics().
    precision : int, optional
        Number of decimal places, by default 3

    Returns
    -------
    pd.DataFrame
        DataFrame with formatted columns like "MAE" = "0.123 +/- 0.012"
    """
    formatted_df = df[['Endpoint']].copy()

    for metric in METRICS:
        mean_col = f'mean_{metric}'
        std_col = f'std_{metric}'
        formatted_df[metric] = df.apply(
            lambda row: f"{row[mean_col]:.{precision}f} +/- {row[std_col]:.{precision}f}",
            axis=1
        )

    return formatted_df


print("Evaluation helpers loaded.")

In [ ]:
# ---------------------------------------------------------------------------
# Evaluate submission on LB / Private / All splits
# ---------------------------------------------------------------------------

# Load ground truth (includes is_leaderboard flag)
gt_df = pd.read_csv(TEST_CSV)
submission = pd.read_csv("submission.csv")

gt_lb      = gt_df[gt_df["is_leaderboard"] == 1].copy()
gt_private = gt_df[gt_df["is_leaderboard"] == 0].copy()
gt_all     = gt_df.copy()

print(f"Split sizes — LB: {len(gt_lb)}, Private: {len(gt_private)}, All: {len(gt_all)}")
print("Running bootstrap evaluation (1000 samples per split)...\n")

splits = {
    "LB":      gt_lb,
    "Private": gt_private,
    "All":     gt_all,
}

split_results = {}
for split_name, gt_subset in splits.items():
    print(f"  Evaluating {split_name}...")
    sub_subset = submission[submission["Molecule Name"].isin(gt_subset["Molecule Name"])].copy()
    split_results[split_name] = calculate_metrics(sub_subset, gt_subset)

# ------------------------------------------------------------------
# Print formatted metrics per split
# ------------------------------------------------------------------
for split_name in ["LB", "Private", "All"]:
    formatted = format_metrics_table(split_results[split_name])
    print(f"\n{'=' * 100}")
    print(f"  {split_name} Split  (mean +/- bootstrap std, 1000 samples)")
    print(f"{'=' * 100}")
    print(formatted.to_string(index=False))

# ------------------------------------------------------------------
# Highlight MA-RAE per split
# ------------------------------------------------------------------
print(f"\n{'—' * 60}")
print("Macro-Averaged RAE per Split:")
for split_name in ["LB", "Private", "All"]:
    df = split_results[split_name]
    ma = df[df["Endpoint"] == "Macro Average"].iloc[0]
    print(f"  MA-RAE ({split_name:>7s}): {ma['mean_RAE']:.3f} +/- {ma['std_RAE']:.3f}")

In [ ]:
# ---------------------------------------------------------------------------
# Scatter plots: Predicted vs True for each endpoint (3x3 grid)
# Values are in log10(y+1) space (except LogD which is as-is)
# ---------------------------------------------------------------------------

gt_df = pd.read_csv(TEST_CSV)
submission = pd.read_csv("submission.csv")

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes_flat = axes.flatten()

for i, ept in enumerate(EVAL_ENDPOINTS):
    ax = axes_flat[i]
    y_pred, y_true = get_point_predictions_with_truth(submission, gt_df, ept)

    ax.scatter(y_true, y_pred, alpha=0.3, s=10, color="steelblue")

    # Diagonal reference line
    lo = min(y_true.min(), y_pred.min())
    hi = max(y_true.max(), y_pred.max())
    margin = (hi - lo) * 0.05
    ax.plot([lo - margin, hi + margin], [lo - margin, hi + margin],
            "r--", alpha=0.7, lw=1)

    # R² annotation
    r2 = r2_score(y_true, y_pred) if np.nanstd(y_true) > 0 else np.nan
    ax.annotate(
        f"R² = {r2:.3f}", xy=(0.05, 0.95), xycoords="axes fraction",
        fontsize=11, ha="left", va="top",
        bbox=dict(boxstyle="round,pad=0.3", facecolor="wheat", alpha=0.5),
    )

    label = "log₁₀(y + 1)" if ept.lower() != "logd" else "value"
    ax.set_xlabel(f"True ({label})")
    ax.set_ylabel(f"Predicted ({label})")
    ax.set_title(ept, fontsize=12)

plt.suptitle("Predicted vs True — Test Set (All)", fontsize=14, y=1.01)
plt.tight_layout()
plt.show()